# Convolutional Neural Network (CNN)

## Convolution
Imagine that you are rolling two fair dice and the outcome of each roll is a probability distribution:
$$
f(x)=g(y)=
\begin{cases}
\frac{1}{6}, & \text{if }x\in\{1,2,3,4,5,6\} \\
0, & \text{otherwise}
\end{cases}
$$
The distribution $g$ may be different from $f$ if $g$ has certain weights on certain values. Then the probability of rolling 4 in total could be
$$
f(1)g(3) = \frac{1}{36} .
$$
To find the *total likelihood* of two dice summing 4, we have consider all possible values of the two dice, that is, we consider all possible partitions of 4. Thus, the probability of two dice summing 4 is 
$$
f(1)g(3)+f(2)g(2)+f(3)g(1) = \sum_{x+y=4}f(x)g(y) = \frac{1}{12}.
$$
This is exactly a convolution. In general, a convolution evaluated at $c$ is defined as
$$
(f*g)(c)\dot{=}\sum_{x+y=c}f(x)g(y),
$$
or
$$
(f*g)(c)=\sum_x f(x)g(c-x). 
$$
For continuous probability distribution, we replace the sum by a integral over the domain
$$
(f*g)(c)=\int_{\mathbb{R}} f(x)g(c-x) dx.
$$

The convolutional layers in CNN is an application of convolutions for matrices.

**Definition** Let $M\in\mathbb{R}^{n_1\times n_2}$ and $K\in\mathbb{R}^{m_1\times m_2}$ such that $m_1\leq n_1$, $m_2\leq n_2$. The **convolution** of $M$ and $K$ is denoted by $Y*K$ with entries
$$
[M*K]_{i,j}=\sum_{k=1}^{m_1}\sum_{l=1}^{m_2} K_{k,l}Y_{i+m_1-k,j+m_2-l}
$$
for $1\leq i\leq n_1-m_1+1$ and $1\leq j\leq n_2-m_2+1$. Here $M$ is called the input and $K$ is called the **kernel**. Typically, the kernel is usually square, say $m\times m$, and it's a hyperparameter of $m^2$ parameters of CNN.

**Example** Suppose we have a data matrix
$$
M=
\left[\begin{matrix}
1 & 5 & -2 & 0 & 2 \\
3 & 8 & 7 & 1 & 0 \\
−1 & 0 & 1 & 2 & 3 \\
4 & 2 & 1 & −1 & 2 \\
\end{matrix}\right].
$$
and a kernel matrix
$$
K=
\left[\begin{matrix}
1 & 2 & 3 \\
4 & 5 & 6 \\
7 & 8 & 9 \\
\end{matrix}\right].
$$
The definition might look confusing at a glance, but the computation of entries can be roughly said that each element of $M$ is multiplied by the entry of flipped up and down, left and right kernel $K$. 
$$
[M*K]_{1,1}=
\left[\begin{matrix}
1\cdot 9 & 5\cdot 8 & -2\cdot 7 & 0 & 2 \\
3\cdot 6 & 8\cdot 5 & 7\cdot 4 & 1 & 0 \\
−1\cdot 3 & 0\cdot 2 & 1\cdot 1 & 2 & 3 \\
4 & 2 & 1 & −1 & 2 \\
\end{matrix}\right]
=9+40-14+18+40+28-3+0+1=119
$$

## Padding and Stride
The convolution operation will reduce the dimension of the input matrix. Sometimes we hope that the dimension is not reduced quickly by convolutions or even retain the original size. We can use **(zero) padding** $p\in\mathbb{N}$ to keep the shape of matrix. For $p=1$, the matrix $M$ becomes
$$
M=
\left[\begin{matrix}
0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 5 & -2 & 0 & 2 & 0 \\
0 & 3 & 8 & 7 & 1 & 0 & 0 \\
0 & −1 & 0 & 1 & 2 & 3 & 0 \\
0 & 4 & 2 & 1 & −1 & 2 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 
\end{matrix}\right].
$$
We can also set an integer **stride**, which means how far to move the kernel. Usually, the kernel moves from left to right, top to down, and performs a convolution with the corresponding submatrix. For example, when the stride is 2, the kernel moves as the following:

![stride=2](images/stride.png)

![stride=2](CNN/images/stride.png)

(red: before, green: after)

In general, a stride $s>1$ reduces the dimension of the output of a convolution to
$$
\left(\frac{n_1-m+1}{s}\right)\times\left(\frac{n_2-m+1}{s}\right).
$$
If we have $N$ filters, then there are $N\cdot m^2$ variables and the output size of a convolution is
$$
\left(\frac{n_1+2p-m}{s}\times\frac{n_2+2p-m}{s}\right)\times N
$$
where the results for all M filters are stacked.

## 3-dimensional Convolutional Layers
Now we introduce the 3-dimensional convolutional layers. Let the input tensor (yes, it's no longer a matrix; it's a tensor) be of size $n_1\times n_2\times n_3$, and the kernel of size $m\times m\times n_3$, i.e. the depth is the same.